In [40]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import plotly.express as px
import os
from datetime import datetime
import time
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from statistics import stdev

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [4]:

def load_data(fx_pair,years):
    """
    years: list of years, or int of year
    fx_pair: "EURUSD", "GBPCHF", etc.
    """
    if type(years) == int: 
        years = [years]
    
    df = None
    for year in years:
        temp_df = pd.read_csv("./Cleaned_OHLC_FOREX_Data/{}/{}_{}_1min.csv".format(fx_pair, fx_pair,  year)).drop(columns =['Unnamed: 0'])
        
        if type(df) == None:
            df = temp_df.copy()
        else: 
            df = pd.concat([df, temp_df], axis=0)
    return df


df = load_data("EURUSD",2012)
df.head(20)

df

,Date_time,Open,High,Low,Close
0,2012-01-02 02:00:00,1.29324,1.29381,1.29324,1.29332
1,2012-01-02 02:01:00,1.29326,1.29345,1.29275,1.29341
2,2012-01-02 02:02:00,1.29342,1.29344,1.29341,1.29343
3,2012-01-02 02:03:00,1.29342,1.29342,1.29335,1.29336
4,2012-01-02 02:04:00,1.29324,1.29347,1.29320,1.29346
...,...,...,...,...,...
373015,2012-12-31 16:54:00,1.31902,1.31938,1.31899,1.31930
373016,2012-12-31 16:55:00,1.31928,1.31937,1.31918,1.31934
373017,2012-12-31 16:56:00,1.31935,1.31938,1.31924,1.31925
373018,2012-12-31 16:57:00,1.31927,1.31949,1.31925,1.31944


In [157]:
# Checking consistency of times fpr 1 minute data. 

df = load_data("EURGBP",2015)

prev_date = 0
prev_date_int = 0

for idx,date in enumerate(df['Date_time']):
    if int(date[11:17].replace(":","")) - prev_date_int == 1 or int(date[11:17].replace(":","")) - prev_date_int == 41 or int(date[11:17].replace(":","")) - prev_date_int == -2359:
        pass
    else:
        print(f"Prev Date: {prev_date}   Cur Date: {date}")
        print(int(date[11:17].replace(":","")) - prev_date_int)
        
    prev_date_int = int(date[11:17].replace(":",""))
    prev_date = date

Prev Date: 0   Cur Date: 2015-01-01 13:00:00
1300
Prev Date: 2015-01-01 14:18:00   Cur Date: 2015-01-01 17:00:00
282
Prev Date: 2015-04-03 16:56:00   Cur Date: 2015-04-05 17:00:00
44
Prev Date: 2015-05-01 16:59:00   Cur Date: 2015-05-03 17:03:00
44
Prev Date: 2015-07-10 16:58:00   Cur Date: 2015-07-12 17:00:00
42
Prev Date: 2015-08-21 13:16:00   Cur Date: 2015-08-21 13:58:00
42
Prev Date: 2015-08-28 16:59:00   Cur Date: 2015-08-30 17:01:00
42
Prev Date: 2015-09-25 16:59:00   Cur Date: 2015-09-27 17:01:00
42
Prev Date: 2015-10-16 16:59:00   Cur Date: 2015-10-18 17:06:00
47
Prev Date: 2015-11-06 16:59:00   Cur Date: 2015-11-08 17:01:00
42
Prev Date: 2015-12-04 16:59:00   Cur Date: 2015-12-06 17:01:00
42
Prev Date: 2015-12-24 13:59:00   Cur Date: 2015-12-27 17:08:00
349


In [176]:
# Compress data 

def compress_df(df , fx_pair):
    """
    df: dataframe of cleaned OHLC data
    candle_length : int in mins describing candle size
    future_candles: int describing how many candles in the future to predict.
    fx_pair : "EURUSD", "GBPCHF", etc...
    spread  : constant subtracted from the absolute value of ROC.
    candle_length * future_candles == future_prediction_time
    """
    
    temp_high = []
    temp_low = []

    counter = 0
    
    datetime_list = []
    open_list = []
    high_list = []
    low_list  = []
    close_list = []
    
    # These bad boiz should speed things up.
    all_datetime = [i for i in df['Date_time']]
    all_low = [i for i in df['Low']]
    all_open = [i for i in df['Open']]
    all_close = [i for i in df['Close']]
    
    # Start at values of 10 minutes variables
    
    started_yet = False
    
    # Aggregate candles.
    for idx,cur_high in enumerate(df['High'].copy()):
        # Ensuring the first start time is a minute counter divisible by 10.
        if not started_yet:
            if all_datetime[idx][15:16] != "0": 
                continue
            else:
                started_yet = True
                counter = 0
                      
        if (all_datetime[idx][15:16] != "0") and (counter == 0):
            continue
        
        counter+= 1
        cur_low = all_low[idx]

        temp_high.append(cur_high)
        temp_low.append(cur_low)
        
        if all_datetime[idx][15:16] == "0":
            # Append time and open
            if counter == 2:
                datetime_list = datetime_list[:-1]
                open_list = open_list[:-1]
            
            datetime_list.append(all_datetime[idx])
            open_list.append(round(all_open[idx],5))


        elif (counter== 10) or idx == len(all_datetime)-1 or (all_datetime[idx+1][15:16] == "0"):
            #print(f"idx: {idx}  counter: {counter}")
            # Append high,low,close , reset counter
            counter = 0
            high_list.append(round(max(temp_high),5))
            low_list.append(round(min(temp_low),5))
            temp_high = []
            temp_low = []
            close_list.append(round(all_close[idx],5))
            
            
            #print(f"Date:{datetime_list[-1]}  Open: {open_list[-1]}  High: {high_list[-1]}  Low: {low_list[-1]}  Close: {close_list[-1]}")
            
        if abs(len(open_list) - len(close_list)) == 2:
            print(f"len(open_list): {len(open_list)}  len(close_list): {len(close_list)}")
            print("counter: ",counter)
            print("all_datetime[idx][15:16]: ", all_datetime[idx][15:16])
            print("datetime: ", all_datetime[idx])
            print("idx: ", idx)
            print()
            
#         if (len(open_list) != len(close_list)) and 368044 >= idx >= 368019:
#             print(f"len(open_list): {len(open_list)}  len(close_list): {len(close_list)}")
#             print("counter: ",counter)
#             print("all_datetime[idx][15:16]: ", all_datetime[idx][15:16])
#             print("idx: ", idx)
#             print()
            
#     if len(open_list) > len(close_list):
#         print(f"len(open_list): {len(open_list)}  len(close_list): {len(close_list)}")
#         open_list = open_list[:-1]
#         datetime_list = datetime_list[:-1]
        
#     elif len(open_list) < len(close_list):
#         print(f"len(open_list): {len(open_list)}  len(close_list): {len(close_list)}")
#         close_list = close_list[:-1]
#         high_list = high_list[:-1]
#         low_list = low_list[:-1]
        
#     print("datetime_list len ", len(datetime_list))
    print("open_list length  ", len(open_list))
#     print("high_list length  ", len(high_list))
#     print("low_list length   ", len(low_list))
    print("close_list length ", len(close_list))
        
    df = pd.DataFrame({'Date_time':datetime_list,
                       'Open' : open_list,
                       'High' : high_list,
                       'Low'  : low_list,
                       'Close': close_list})
    return df

fx_pair = "EURGBP"
df = load_data(fx_pair, 2016)
new_df = compress_df(df, fx_pair)
new_df.tail(15)

open_list length   37333
close_list length  37333


,Date_time,Open,High,Low,Close
37318,2016-12-30 14:30:00,0.85492,0.85521,0.85481,0.85514
37319,2016-12-30 14:40:00,0.85516,0.85559,0.85516,0.85523
37320,2016-12-30 14:50:00,0.85525,0.85531,0.85454,0.85484
37321,2016-12-30 15:00:00,0.85489,0.85505,0.85433,0.85459
37322,2016-12-30 15:10:00,0.85458,0.85463,0.85417,0.85419
37323,2016-12-30 15:20:00,0.85418,0.85462,0.85415,0.85427
37324,2016-12-30 15:30:00,0.85429,0.85429,0.85378,0.85384
37325,2016-12-30 15:40:00,0.85385,0.85442,0.85362,0.85426
37326,2016-12-30 15:50:00,0.85426,0.85451,0.85384,0.85403
37327,2016-12-30 16:00:00,0.85395,0.85423,0.85389,0.85412


# Checking for inconsistentcies

In [188]:
def print_ohlc_row(idx, prev_close, _open, close, inconsist):
    print(f"IDX: {idx}  Previous Close: {prev_close}  Open: {_open}  Close: {close}  Status: {inconsist}")
            

def report_inconsistencies_original_data(ohlc_df, verbose = False ):
    
    fluctuations = {"real":[] , "inconsistent":[] , 
                    "real_close_price_change": [] , "inconsistent_close_price_change":[] }
        
    prev_close = ohlc_df.loc[0,'Close']
    for idx,close in tqdm(enumerate(ohlc_df['Close']), total = len(ohlc_df)):
        date = str(ohlc_df.loc[idx,'Date_time'])[8:10]
        
        if abs(close - prev_close) >= 0.0010:
            
            if (int(date) > int(prev_close)+1) or (int(date) < int(prev_date)-21):
                # If the difference in time is due to a weekend, or monthly change
                # This only activates if a price fluctuation in price is found.
                prev_date = date
                prev_close = close
                
                # Add it as a real fluctuation, as this also happens in the market
                fluctuations['real'].append(idx)
                if verbose:
                    print_ohlc_row(idx, prev_close, _open, close, "Consistent")
                    
                continue
                
            _open = ohlc_df.loc[idx,'Open']
            if abs(_open - prev_close) >= 0.0010:
                # The difference of open and close price is greathe than 15 pips.
                fluctuations['inconsistent'].append(idx)
                fluctuations['inconsistent_close_price_change'].append(abs(close - prev_close))
                if verbose:
                    print_ohlc_row(idx, prev_close, _open, close, "Inconsistent")
            else:
                # The difference of open and close price is not greater than 15 pips.
                fluctuations['real'].append(idx)
                fluctuations['real_close_price_change'].append(abs(close - prev_close))
                if verbose:
                    print_ohlc_row(idx, prev_close, _open, close, "Consistent")
                    
        
        prev_date = date
        prev_close = close
            
    return fluctuations


def report_inconsistency_data(fluctuations, year_string, total_df_len):
    
    if len(fluctuations['inconsistent_close_price_change']) == 0:
        fluctuations['inconsistent_close_price_change'].append(0)
        fluctuations['inconsistent_close_price_change'].append(0)
    
    r_mean = sum(fluctuations['real_close_price_change']) / len(fluctuations['real_close_price_change'])
    i_mean = sum(fluctuations['inconsistent_close_price_change']) / len(fluctuations['inconsistent_close_price_change'])
    
    r_stdev = stdev(fluctuations['real_close_price_change'])
    i_stdev = stdev(fluctuations['inconsistent_close_price_change'])
    
    r_max = max(fluctuations['real_close_price_change'])
    r_min = min(fluctuations['real_close_price_change'])
    i_max = max(fluctuations['inconsistent_close_price_change'])
    i_min = min(fluctuations['inconsistent_close_price_change'])
    
    print("A price fluctuation is defined as a change of 10 pips from the ")
    print("close price using 1 minute OHLC data. If the past close price, is not within 10 pips")
    print("of the open price, it is considered inconsistent.\n")
    print("")
    print("Price fluctuation statistics {}".format(year_string))
    print(f"Number of rows in dataframe            : {total_df_len}")
    print(f"Number of real fluctuations            : {len(fluctuations['real'])}")
    print(f"Number of inconsistent fluctuations    : {len(fluctuations['inconsistent'])}")
    print("")
    print(f"Mean change in close of real fluctuations                             : {r_mean:.5f}")
    print(f"Mean change in close of of inconsistent fluctuations                  : {i_mean:.5f}")
    print("")
    print(f"Standard deviation of change in close of real fluctuations            : {r_stdev:.5f}")
    print(f"Standard deviation of change in close of of inconsistent fluctuations : {i_stdev:.5f}")
    print("")
    print(f"Min, Max of change in close of real fluctuations             : {r_min:.5f}  {r_max:.5f}")
    print(f"Min, Max of change in close of of inconsistent fluctuations  : {i_min:.5f}  {i_max:.5f}")    
    
    
fx_pair = "EURUSD"
for year in range(2012,2020):
    df = load_data(fx_pair, year)
    new_df = compress_df(df, fx_pair)
    if year == 2012:
        total_df = new_df
    else: 
        total_df = pd.concat([total_df, new_df], axis=0) 
          
total_df.reset_index(inplace = True, drop = True)
          
fluctuations = report_inconsistencies_original_data(total_df, False)
report_inconsistency_data(fluctuations, "2012-2019", len(total_df))

open_list length   37302
close_list length  37302
open_list length   37197
close_list length  37197
open_list length   37139
close_list length  37139
open_list length   37280
close_list length  37280
open_list length   37340
close_list length  37340
open_list length   37242
close_list length  37242
open_list length   37355
close_list length  37355
open_list length   37271
close_list length  37271



A price fluctuation is defined as a change of 10 pips from the 
close price using 1 minute OHLC data. If the past close price, is not within 10 pips
of the open price, it is considered inconsistent.


Price fluctuation statistics 2012-2019
Number of rows in dataframe            : 298126
Number of real fluctuations            : 14417
Number of inconsistent fluctuations    : 5

Mean change in close of real fluctuations                             : 0.00154
Mean change in close of of inconsistent fluctuations                  : 0.00506

Standard deviation of change in close of real fluctuations            : 0.00074
Standard deviation of change in close of of inconsistent fluctuations : 0.00560

Min, Max of change in close of real fluctuations             : 0.00100  0.00769
Min, Max of change in close of of inconsistent fluctuations  : 0.00113  0.01476


In [184]:
fx_pair = "EURUSD"
year = 2020
new_df = compress_df(load_data(fx_pair, year), fx_pair)
fluctuations = report_inconsistencies_original_data(new_df, False)
report_inconsistency_data(fluctuations, "2020", len(new_df))

open_list length   37377
close_list length  37377



A price fluctuation is defined as a change of 10 pips from the 
close price using 1 minute OHLC data. If the past close price, is not within 10 pips
of the open price, it is considered inconsistent.


Price fluctuation statistics 2020
Number of rows in dataframe            : 37377
Number of real fluctuations            : 1919
Number of inconsistent fluctuations    : 0

Mean change in close of real fluctuations                             : 0.00151
Mean change in close of of inconsistent fluctuations                  : 0.00000

Standard deviation of change in close of real fluctuations            : 0.00049
Standard deviation of change in close of of inconsistent fluctuations : 0.00000

Min, Max of change in close of real fluctuations             : 0.00102  0.00343
Min, Max of change in close of of inconsistent fluctuations  : 0.00000  0.00000


In [179]:
for fx_pair in ['EURUSD','EURGBP','GBPUSD']:
    for year in range(2020,2021):
        print("Year: {}  FX-pair: {} ".format(year,fx_pair))
        path = "Cleaned_OHLC_FOREX_Data_10_min/{}/".format(fx_pair)
        if not os.path.exists(path):
            os.makedirs(path)
        path_n_name = path + "{}_{}_10min.csv".format(fx_pair,year)
        df = load_data(fx_pair , year)
        compress_df(df , fx_pair).to_csv(path_n_name)

Year: 2020  FX-pair: EURUSD 
open_list length   37377
close_list length  37377
Year: 2020  FX-pair: EURGBP 
open_list length   37347
close_list length  37347
Year: 2020  FX-pair: GBPUSD 
open_list length   37360
close_list length  37360
